#Install Libraries

In [1]:
!pip install openai tiktoken matplotlib  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.1 MB/s eta 0:00:00


#Initialize Packages & setting up the OpenAI Key

In [2]:
#Define OpenAI API keys

import json
import openai
import os

api_key ="sk-***"
openai.api_key = api_key
os.environ['OPENAI_API_KEY'] = api_key

#Data Validation Process

In [4]:
# import numpy as np
# import tiktoken
# import openai
# import json

# # Constants
# MAX_TOKENS_PER_EXAMPLE = 4096
# TARGET_EPOCHS = 2
# MIN_TARGET_EXAMPLES = 20
# MAX_TARGET_EXAMPLES = 50

# # Initialize the token encoding model
# encoding = tiktoken.get_encoding("cl100k_base")

# # ---- Token Counting Utilities ----

# def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
#     """
#     Calculate the total number of tokens in a list of messages.

#     Parameters:
#     messages (list): A list of dictionaries containing messages with roles and content.
#     tokens_per_message (int): The base number of tokens for each message. Default is 3.
#     tokens_per_name (int): Number of tokens for each name in the message. Default is 1.

#     Returns:
#     int: The total number of tokens for the list of messages.
#     """

#     # Initialize the total token count.
#     num_tokens = 0

#     # Iterate through each message in the list of messages.
#     for message in messages:
#         # Add the base tokens_per_message to the total count.
#         num_tokens += tokens_per_message

#         # Iterate through key-value pairs in the message.
#         for key, value in message.items():
#             # Add the length of the encoded value to the total count.
#             num_tokens += len(encoding.encode(value))

#             # Check if the key is "name" and add tokens_per_name if it is.
#             if key == "name":
#                 num_tokens += tokens_per_name

#     # Add 3 additional tokens for message separation.
#     num_tokens += 3

#     # Return the total number of tokens for the list of messages.
#     return num_tokens


# def num_assistant_tokens_from_messages(messages):
#       """
#     Calculate the number of tokens in assistant's messages within a list of messages.

#     Parameters:
#     messages (list): A list of dictionaries containing messages with roles and content.

#     Returns:
#     int: The number of tokens used in assistant's responses.
#     """
#     # Calculate the number of tokens in assistant messages within a list of messages.
#     num_tokens = 0
#     for message in messages:
#         if message["role"] == "assistant":
#             num_tokens += len(encoding.encode(message["content"]))
#     return num_tokens

# def print_distribution(values, name):
#     """
#     Print statistical distribution information for a list of values.

#     Parameters:
#     values (list): List of numerical values for which distribution is calculated.
#     name (str): Descriptive name for the list of values.
#     """
#     # Print distribution statistics for a list of values.
#     print(f"\n#### Distribution of {name}:")
#     print(f"min / max: {min(values)}, {max(values)}")
#     print(f"mean / median: {np.mean(values)}, {np.median(values)}")
#     print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# # ---- Additional Check: Duplicate Conversations ----

# def check_for_duplicates(dataset):
#     """
#     Check for duplicate conversations in the dataset.

#     Parameters:
#     dataset (list): List of conversation data.

#     Prints:
#     Number of duplicate conversations found in the dataset.
#     """
#     # Check for duplicate conversations in the dataset.
#     unique_conversations = set()
#     duplicate_count = 0

#     for ex in dataset:
#         serialized = json.dumps(ex, sort_keys=True)
#         if serialized in unique_conversations:
#             duplicate_count += 1
#         else:
#             unique_conversations.add(serialized)

#     print(f"\n### Duplicate Conversations Check ###")
#     print(f"Number of duplicate conversations: {duplicate_count}")

# # ---- Data Warnings and Token Counts ----

# def analyze_data_and_tokens(dataset):
#     """
#     Perform a comprehensive analysis on the dataset to ensure its quality.

#     Parameters:
#     dataset (list): List of conversation data.

#     Prints:
#     Various warnings, token counts, and other statistics for the dataset.
#     """
#     # Analyze the dataset for data warnings and token counts.
#     n_missing_system = 0
#     n_missing_user = 0
#     n_messages = []
#     convo_lens = []
#     assistant_message_lens = []
#     for ex in dataset:
#         messages = ex["messages"]
#         if not any(message["role"] == "system" for message in messages):
#             n_missing_system += 1
#         if not any(message["role"] == "user" for message in messages):
#             n_missing_user += 1
#         n_messages.append(len(messages))
#         convo_lens.append(num_tokens_from_messages(messages))
#         assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

#     print(f"\n### Data Analysis and Token Counts ###")
#     print("Num examples missing system message:", n_missing_system)
#     print("Num examples missing user message:", n_missing_user)
#     print_distribution(n_messages, "num_messages_per_example")
#     print_distribution(convo_lens, "num_total_tokens_per_example")
#     print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")

#     n_too_long = sum(l > 4096 for l in convo_lens)
#     print(f"{n_too_long} examples may be over the 4096 token limit and will be truncated during fine-tuning")

# # ---- Cost Estimation ----

# def estimate_cost(dataset, price_per_1k_token=0.0005):
#     """
#     Estimate the cost of fine-tuning the model based on dataset size and token count.

#     Parameters:
#     dataset (list): List of conversation data.
#     price_per_1k_token (float): Cost per 1,000 tokens for training the model.

#     Prints:
#     Estimated cost for training the model.
#     """
#     # Estimate the cost of training using the dataset.
#     n_epochs = TARGET_EPOCHS
#     n_train_examples = len(dataset)

#     if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
#         n_epochs = min(MAX_TARGET_EXAMPLES, MIN_TARGET_EXAMPLES // n_train_examples)
#     elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
#         n_epochs = max(MIN_TARGET_EXAMPLES, MAX_TARGET_EXAMPLES // n_train_examples)

#     convo_lens = [num_tokens_from_messages(ex["messages"]) for ex in dataset]
#     n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)

#     print(f"\n### Cost Estimation ###")
#     print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training.")
#     print(f"By default, you'll train for {n_epochs} epochs on this dataset.")
#     print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens.")

#     # Cost Estimation
#     total_tokens = n_epochs * n_billing_tokens_in_dataset
#     estimated_cost = (total_tokens / 1000) * price_per_1k_token

#     print(f"Estimated cost for this training is approximately ${estimated_cost}")


# # ---- Execution Pipeline ----

# if __name__ == "__main__":
#     data_path = "/content/chat_data.jsonl"

#     # Load the dataset
#     with open(data_path, 'r', encoding='utf-8') as f:
#         dataset = [json.loads(line) for line in f]

#     # Phase 2: Token Counting and Cost Estimation
#     analyze_data_and_tokens(dataset)
#     estimate_cost(dataset)
#     check_for_duplicates(dataset)


In [3]:
import numpy as np
import tiktoken
import openai
import json

# Constants
MAX_TOKENS_PER_EXAMPLE = 4096
TARGET_EPOCHS = 2
MIN_TARGET_EXAMPLES = 20
MAX_TARGET_EXAMPLES = 50

# Initialize the token encoding model
encoding = tiktoken.get_encoding("cl100k_base")

# ---- Token Counting Utilities ----

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    """
    Calculate the total number of tokens in a list of messages.

    Parameters:
    messages (list): A list of dictionaries containing messages with roles and content.
    tokens_per_message (int): The base number of tokens for each message. Default is 3.
    tokens_per_name (int): Number of tokens for each name in the message. Default is 1.

    Returns:
    int: The total number of tokens for the list of messages.
    """

    # Initialize the total token count.
    num_tokens = 0

    # Iterate through each message in the list of messages.
    for message in messages:
        # Add the base tokens_per_message to the total count.
        num_tokens += tokens_per_message

        # Iterate through key-value pairs in the message.
        for key, value in message.items():
            # Add the length of the encoded value to the total count.
            num_tokens += len(encoding.encode(value))

            # Check if the key is "name" and add tokens_per_name if it is.
            if key == "name":
                num_tokens += tokens_per_name

    # Add 3 additional tokens for message separation.
    num_tokens += 3

    # Return the total number of tokens for the list of messages.
    return num_tokens
def num_assistant_tokens_from_messages(messages):
    """
    Calculate the number of tokens in assistant's messages within a list of messages.

    Parameters:
    messages (list): A list of dictionaries containing messages with roles and content.

    Returns:
    int: The number of tokens used in assistant's responses.
    """
    num_tokens = 0
    for message in messages:  # iterate through each message in the list
        if message["role"] == "assistant":  # if the role of message is 'assistant'
            num_tokens += len(encoding.encode(message["content"]))  # encode the content and count its length
    return num_tokens

def print_distribution(values, name):
    """
    Print statistical distribution information for a list of values.

    Parameters:
    values (list): List of numerical values for which distribution is calculated.
    name (str): Descriptive name for the list of values.
    """
    # print the statistical distribution of the given 'values'
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")  # print minimum and maximum of the 'values'
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")  # print mean and median of the 'values'
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")  # print 10th and 90th percentiles

def check_for_duplicates(dataset):
    """
    Check for duplicate conversations in the dataset.

    Parameters:
    dataset (list): List of conversation data.

    Prints:
    Number of duplicate conversations found in the dataset.
    """
    unique_conversations = set()  # a set to store unique conversations
    duplicate_count = 0  # counter for duplicate conversations

    for ex in dataset:  # iterate through each example in the dataset
        serialized = json.dumps(ex, sort_keys=True)  # serialize the example
        if serialized in unique_conversations:  # if the serialized example is already in the set
            duplicate_count += 1  # increment the duplicate counter
        else:
            unique_conversations.add(serialized)  # if not, add it to the set

    print(f"\n### Duplicate Conversations Check ###")
    print(f"Number of duplicate conversations: {duplicate_count}")

def analyze_data_and_tokens(dataset):
    """
    Perform a comprehensive analysis on the dataset to ensure its quality.

    Parameters:
    dataset (list): List of conversation data.

    Prints:
    Various warnings, token counts, and other statistics for the dataset.
    """
    n_missing_system = 0  # counter for missing system messages
    n_missing_user = 0  # counter for missing user messages
    n_messages = []  # store number of messages in each conversation
    convo_lens = []  # store the total number of tokens in each conversation
    assistant_message_lens = []  # store the total number of tokens in assistant messages in each conversation

    for ex in dataset:  # iterate through each example in the dataset
        messages = ex["messages"]
        if not any(message["role"] == "system" for message in messages):  # check for missing system message
            n_missing_system += 1
        if not any(message["role"] == "user" for message in messages):  # check for missing user message
            n_missing_user += 1
        n_messages.append(len(messages))  # append the number of messages
        convo_lens.append(num_tokens_from_messages(messages))  # append the total number of tokens
        assistant_message_lens.append(num_assistant_tokens_from_messages(messages))  # append the total number of tokens in assistant messages

    # print the results of the analysis
    print(f"\n### Data Analysis and Token Counts ###")
    print("Num examples missing system message:", n_missing_system)
    print("Num examples missing user message:", n_missing_user)
    print_distribution(n_messages, "num_messages_per_example")
    print_distribution(convo_lens, "num_total_tokens_per_example")
    print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")

    n_too_long = sum(l > 4096 for l in convo_lens)  # count of conversations exceeding the token limit
    print(f"{n_too_long} examples may be over the 4096 token limit and will be truncated during fine-tuning")

def estimate_cost(dataset, price_per_1k_token=0.0005):
    """
    Estimate the cost of fine-tuning the model based on dataset size and token count.

    Parameters:
    dataset (list): List of conversation data.
    price_per_1k_token (float): Cost per 1,000 tokens for training the model.

    Prints:
    Estimated cost for training the model.
    """
    n_epochs = TARGET_EPOCHS  # target number of epochs for training
    n_train_examples = len(dataset)  # total number of examples in the dataset

    # adjust the number of epochs based on the number of training examples
    if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
        n_epochs = min(MAX_TARGET_EXAMPLES, MIN_TARGET_EXAMPLES // n_train_examples)
    elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
        n_epochs = max(MIN_TARGET_EXAMPLES, MAX_TARGET_EXAMPLES // n_train_examples)

    convo_lens = [num_tokens_from_messages(ex["messages"]) for ex in dataset]  # total number of tokens in each conversation
    n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)  # total number of tokens to be billed

    # print the cost estimation
    print(f"\n### Cost Estimation ###")
    print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training.")
    print(f"By default, you'll train for {n_epochs} epochs on this dataset.")
    print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens.")

    total_tokens = n_epochs * n_billing_tokens_in_dataset  # total number of tokens to be billed
    estimated_cost = (total_tokens / 1000) * price_per_1k_token  # calculate the estimated cost

    print(f"Estimated cost for this training is approximately ${estimated_cost}")

# ---- Execution Pipeline ----

if __name__ == "__main__":
    data_path = "/content/chat_data.jsonl"

    # Load the dataset
    with open(data_path, 'r', encoding='utf-8') as f:
        dataset = [json.loads(line) for line in f]

    # Phase 2: Token Counting and Cost Estimation
    analyze_data_and_tokens(dataset)
    estimate_cost(dataset)
    check_for_duplicates(dataset)




### Data Analysis and Token Counts ###
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 50, 79
mean / median: 66.65, 68.0
p5 / p95: 58.5, 74.2

#### Distribution of num_assistant_tokens_per_example:
min / max: 14, 28
mean / median: 22.85, 24.0
p5 / p95: 17.6, 26.1
0 examples may be over the 4096 token limit and will be truncated during fine-tuning

### Cost Estimation ###
Dataset has ~1333 tokens that will be charged for during training.
By default, you'll train for 2 epochs on this dataset.
By default, you'll be charged for ~2666 tokens.
Estimated cost for this training is approximately $0.001333

### Duplicate Conversations Check ###
Number of duplicate conversations: 0


#Upload Chat Data

In [ ]:
# Define the path to the file you want to upload
file_name = "/content/chat_data.jsonl"

# Use OpenAI's File.create method to upload the file for the purpose of fine-tuning
# The file is opened in binary read mode ("rb")
upload_response = openai.File.create(
  file=open(file_name, "rb"),  # Open the file in binary read mode
  purpose='fine-tune'  # Specify that the file is intended for fine-tuning a model
)

# Output the upload response to see the details of the uploaded file
upload_response


<File file id=file-xsa036zTL32wxy5eGSUzjwpE at 0x7d7ec0ccb6f0> JSON: {
  "object": "file",
  "id": "file-xsa036zTL32wxy5eGSUzjwpE",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 8474,
  "created_at": 1695643341,
  "status": "uploaded",
  "status_details": null
}

#Identify uploaded file id

In [ ]:
#Save file name
file_id = upload_response.id
file_id


'file-xsa036zTL32wxy5eGSUzjwpE'

#Initiate the finetuning job

In [ ]:
# Call the FineTuningJob.create method to initiate the fine-tuning process
# Provide the training_file using the previously saved file_id and specify the model to use
openai.FineTuningJob.create(training_file=file_id, model="gpt-3.5-turbo")


<FineTuningJob fine_tuning.job id=ftjob-3MKXBj9VV4Q5xtUuJkDz0pEn at 0x7d7ec0d199e0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-3MKXBj9VV4Q5xtUuJkDz0pEn",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1695643358,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-Fw23HNdOBixFvYL3rPmDB7Or",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-xsa036zTL32wxy5eGSUzjwpE",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

#Retrieve finetuning jobs for verification

In [ ]:
# List 10 fine-tuning jobs
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x7d7e8bdd7b00> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-3MKXBj9VV4Q5xtUuJkDz0pEn",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1695643358,
      "finished_at": null,
      "fine_tuned_model": null,
      "organization_id": "org-Fw23HNdOBixFvYL3rPmDB7Or",
      "result_files": [],
      "status": "running",
      "validation_file": null,
      "training_file": "file-xsa036zTL32wxy5eGSUzjwpE",
      "hyperparameters": {
        "n_epochs": 5
      },
      "trained_tokens": null,
      "error": null
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-7bLutQbDbVq0KJ0SGikLnoVR",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1695155870,
      "finished_at": 1695156183,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::80bqhDsb",
      "organization_id": "org-Fw23HNdOBixFvYL3rPmDB7Or",
      "result_files": [
        "file-NBHZpOsxP9i16lubc1eBS9CE"
      ],
      

In [ ]:
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x7d7e8bdd6c50> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-3MKXBj9VV4Q5xtUuJkDz0pEn",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1695643358,
      "finished_at": 1695643682,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::82efbhJ6",
      "organization_id": "org-Fw23HNdOBixFvYL3rPmDB7Or",
      "result_files": [
        "file-8WfCUkOWa5vW44C9RTOxg4Ug"
      ],
      "status": "succeeded",
      "validation_file": null,
      "training_file": "file-xsa036zTL32wxy5eGSUzjwpE",
      "hyperparameters": {
        "n_epochs": 5
      },
      "trained_tokens": 6465,
      "error": null
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-7bLutQbDbVq0KJ0SGikLnoVR",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1695155870,
      "finished_at": 1695156183,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::80bqhDsb",
      "organization_id": "org-Fw23HNdOBixFv

#Retrieve the current finetuning job and check for status

In [ ]:
# Using the FineTuningJob.retrieve method to fetch the current status
# of the fine-tuning job with ID "ftjob-3MKXBj9VV4Q5xtUuJkDz0pEn"
openai.FineTuningJob.retrieve("ftjob-3MKXBj9VV4Q5xtUuJkDz0pEn")


<FineTuningJob fine_tuning.job id=ftjob-3MKXBj9VV4Q5xtUuJkDz0pEn at 0x7d7e8b3e6890> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-3MKXBj9VV4Q5xtUuJkDz0pEn",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1695643358,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-Fw23HNdOBixFvYL3rPmDB7Or",
  "result_files": [],
  "status": "running",
  "validation_file": null,
  "training_file": "file-xsa036zTL32wxy5eGSUzjwpE",
  "hyperparameters": {
    "n_epochs": 5
  },
  "trained_tokens": null,
  "error": null
}

#API to check for events associated with the job

In [ ]:
# Cancel a job
# openai.FineTuningJob.cancel("ft-abc123")

# List up to 10 events from a fine-tuning job
openai.FineTuningJob.list_events(id="ftjob-3MKXBj9VV4Q5xtUuJkDz0pEn", limit=10)


<OpenAIObject list at 0x7d7ec0f0d5d0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-1B9qQC6Nteq4T2BfqZLxKgiR",
      "created_at": 1695643362,
      "level": "info",
      "message": "Fine-tuning job started",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-1FJtC9eiUvWW9ZPO2qVvU5oh",
      "created_at": 1695643360,
      "level": "info",
      "message": "Files validated, moving job to queued state",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-HfhhnvQE2MojnrQVw1RtF6XT",
      "created_at": 1695643358,
      "level": "info",
      "message": "Validating training file: file-xsa036zTL32wxy5eGSUzjwpE",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-jaEkBsBfiHZ1xnbl1yHpWyFK",
      "created_at": 1695643358,
      "level

In [ ]:
# List events associated with a fine-tuning job using OpenAI's API.
openai.FineTuningJob.list_events(id="ftjob-3MKXBj9VV4Q5xtUuJkDz0pEn", limit=30)


<OpenAIObject list at 0x7d7e8aef5210> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-yRpD9ORqA46yqPjQJdVQDJH4",
      "created_at": 1695643685,
      "level": "info",
      "message": "The job has successfully completed",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-AmAC95HZdhIazE7kcri4Enlp",
      "created_at": 1695643683,
      "level": "info",
      "message": "New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::82efbhJ6",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-RJXmDbtoBTNvK4OFXg81uFgy",
      "created_at": 1695643666,
      "level": "info",
      "message": "Step 91/100: training loss=0.08",
      "data": {
        "step": 91,
        "train_loss": 0.08288879692554474,
        "train_mean_token_accuracy": 1.0
      },
      "type": "metrics"
    },
    {
      "obj

In [ ]:
# Delete a fine-tuned model (must be an owner of the org the model was created in)
# openai.Model.delete("ftjob-7bLutQbDbVq0KJ0SGikLnoVR")

In [ ]:
content = openai.File.download("file-8WfCUkOWa5vW44C9RTOxg4Ug")
print(content)

b'step,train_loss,train_accuracy,valid_loss,valid_mean_token_accuracy\n1,2.28782,0.68966,,\n2,2.88231,0.42308,,\n3,3.00459,0.55556,,\n4,4.49071,0.5,,\n5,4.28464,0.35,,\n6,4.46689,0.375,,\n7,3.1264,0.44444,,\n8,2.31342,0.66667,,\n9,2.4013,0.46429,,\n10,2.12376,0.56,,\n11,2.64778,0.52174,,\n12,2.03157,0.61538,,\n13,1.64289,0.60714,,\n14,1.91289,0.53846,,\n15,1.70548,0.72727,,\n16,1.25589,0.59259,,\n17,1.59485,0.71429,,\n18,2.11192,0.51852,,\n19,0.98359,0.66667,,\n20,1.33656,0.54545,,\n21,1.63837,0.5625,,\n22,0.73613,0.75,,\n23,1.79567,0.55556,,\n24,0.8778,0.75,,\n25,0.9519,0.77273,,\n26,0.8665,0.76923,,\n27,1.01,0.73077,,\n28,1.10468,0.7037,,\n29,0.68648,0.83333,,\n30,0.60373,0.82759,,\n31,0.58907,0.8,,\n32,1.55252,0.5,,\n33,1.13341,0.77778,,\n34,0.70665,0.80769,,\n35,0.85422,0.67857,,\n36,0.76846,0.73913,,\n37,0.67244,0.68,,\n38,0.81952,0.77273,,\n39,0.5078,0.77778,,\n40,1.09633,0.65,,\n41,0.34753,0.93103,,\n42,0.45934,0.92857,,\n43,0.6221,0.86957,,\n44,0.3977,0.88889,,\n45,0.99037,0.77

In [ ]:
import csv
from tabulate import tabulate  # You may need to install the 'tabulate' library

# Replace the string with your actual output
output_bytes = content

# Decode the bytes into a string
output_string = output_bytes.decode('utf-8')

# Remove the leading 'b' and convert to a string
output_string = output_string[2:].strip()

# Split the string into lines and parse as CSV
lines = output_string.split('\n')
reader = csv.reader(lines)

# Convert CSV data to a list of lists
data = [row for row in reader]

# Print the data as a table
print(tabulate(data, headers="firstrow", tablefmt="grid"))


+------+--------------+------------------+--------------+-----------------------------+
|   ep |   train_loss |   train_accuracy | valid_loss   | valid_mean_token_accuracy   |
+======+==============+==================+==============+=============================+
|    1 |      2.28782 |          0.68966 |              |                             |
+------+--------------+------------------+--------------+-----------------------------+
|    2 |      2.88231 |          0.42308 |              |                             |
+------+--------------+------------------+--------------+-----------------------------+
|    3 |      3.00459 |          0.55556 |              |                             |
+------+--------------+------------------+--------------+-----------------------------+
|    4 |      4.49071 |          0.5     |              |                             |
+------+--------------+------------------+--------------+-----------------------------+
|    5 |      4.28464 |         

In [ ]:
completion = openai.ChatCompletion.create(
  model="ft:gpt-3.5-turbo-0613:personal::82efbhJ6",
  messages=[
    {"role": "system", "content": "You are a Software Engineering Mentor specialized in architecture and high-scale systems"},
    {"role": "user", "content": "What's the role of a load balancer in a high-scale system, and which load balancing algorithms are commonly used?"},
    {"role": "assistant", "content":""}
  ]
)
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "A load balancer distributes incoming traffic across multiple servers to ensure availability and optimize resource utilization. Common algorithms include round-robin and least connections."
}


In [ ]:
# Execute the delete operation for the specified fine-tuned model
# Note: You must be an owner of the organization under which the model was created to perform this action
openai.Model.delete("ft:gpt-3.5-turbo-0613:personal::82efbhJ6")
